In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
url = 'https://sosul.network/series/'

In [5]:
for p in range(1, 100):
    params = {'order_by' : 'review', 'page' : f'{p}'}
    res = requests.get(url, params = params)
    break

In [6]:
soup = BeautifulSoup(res.content, 'html.parser')
a_li = soup.find_all('div', class_='col-6 col-md-4 col-lg-15')

In [7]:
len(a_li)

30

In [8]:
for i in a_li:
    a = i.find('div', class_='product-title')
    path = a.a['href']
    url = 'https://sosul.network' + path
    make_review(url)
    break

내가하면 오리지널 남이하면 파쿠리라는 갓 마인드를 가진작가님의 작품거기에 문피아에 똥꼬를 품
['?filter', 'review&order_by', 'like_count&page', '2']
https://sosul.network/series/190?filter=review&order_by=like_count&page=2
다른 사람들의 리뷰도 사실 크게 틀린 것은 아님 뭐... 개인 차이가 있는 거긴 한데 나도 어느 부분의 공감하는 것도 있음. 다만 이곳의 평은 무섭구나 함. 세계관도 정말 넓긴 한데... 다음 작품에서는 그냥 다른 판타지 처럼 완전히 새로운 판타지 세계를 쓰는 것도 좋을 듯 확실히 작가가 글을 잘 쓰고 어삭하거나 이상한 부분도 크게 없음 비엘은 잘 모르겠으나 읽는 데에 큰 불평을 가져본 적은 없음. 영화로 5섯편 제작된다고도 함 신과 함께 제작사 에서 만든다고 하는데 큰 기대는 안함 그리고 다른 작가가 표절했다는 말이 많은데... 그건 사실 동의하기 조금 어러움 어쩔 수 없이 세계관을 빌려 쓰고 그러면 겹치는 부분이 많은 건 당연할지도 모르고 아마 이전의 작품과 이것역시 비슷한 부분은 많을 지도 모름여튼 그래도 난 기승전결 좋은 작품중 하나인 것 만큼은 확실하다고 생각하는 바임.
['?filter', 'review&order_by', 'like_count&page', '3']
https://sosul.network/series/190?filter=review&order_by=like_count&page=3
이거저거 다 쓰까다 잘 버무려서 잘 읽고 있었는데 갑자기 분위기 똥구멍됨
['?filter', 'review&order_by', 'like_count&page', '4']
https://sosul.network/series/190?filter=review&order_by=like_count&page=4
초반에는 재미있는 요소를 미친듯이 넣고전개속도 긴장감 등은 꽤 뛰어나다 할수있다.그러나 소설 중후반부터는 bl요소를 첨가하는 덕분에 bl빠들

In [ ]:
a

In [ ]:
a.a['href']

In [2]:
import pandas as pd

def make_review(url):
    base_url = url
    while True:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        review_li = soup.find_all('div', class_ = 'comment widget p-cb')
        idx = 0
        for r in review_li:
            b = r.find('div', class_ = 'text').find('p', class_='content-txt').text
            print(b)
            break
        idx += 1
        next_page = soup.find('ul', class_ = "pagination justify-content-center").find_all('li')
        c = next_page[-1].a['href']
        print(c.split('='))
        if c.split('=')[-1] == '#':
            break
        url = base_url + c
        print(url)

In [ ]:
?filter=review&order_by=like_count&page=2